In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
df_vehicle_emergencies = pd.read_csv('/content/df_vehicle_emergencies.csv')
df_questions = pd.read_csv('/content/df_questions.csv')

In [6]:
df_questions

,Category,Questions
0,Flat Tire,What should I do if I get a flat tire?
1,Flat Tire,How do I change a flat tire?
2,Flat Tire,Can I drive with a flat tire?
3,Flat Tire,What’s the quickest way to fix a flat tire?
4,Flat Tire,Where can I get help for a flat tire?
...,...,...
95,Stuck in Mud or Snow,Can I use traction mats to get unstuck?
96,Stuck in Mud or Snow,Should I call for help if my car is stuck?
97,Stuck in Mud or Snow,How do I avoid getting stuck in snow or mud?
98,Stuck in Mud or Snow,Can kitty litter really help get unstuck?


In [7]:
df_vehicle_emergencies

,Category,Response
0,Flat Tire,Use your spare tire and jack to replace the fl...
1,Engine Overheating,"Turn off the air conditioner, pull over, and l..."
2,Dead Battery,Use jumper cables to restart the vehicle or ca...
3,Brake Failure,Shift to a lower gear and use the handbrake gr...
4,Locked Out of Car,Contact a locksmith or your vehicle's roadside...
5,Car Won’t Start,"Check the battery, ignition switch, and fuel l..."
6,Fuel Leak,Turn off the engine immediately and move away ...
7,Blown Fuse,Replace the fuse with one of the same rating f...
8,Smoke from the Hood,"Turn off the vehicle, open the hood cautiously..."
9,Stuck in Mud or Snow,Use traction mats or place sand or kitty litte...


In [8]:
# Combining question and category
df_combined = df_questions.copy()
df_combined = df_combined.rename(columns={'Questions': 'Query'})
df_combined = df_combined.merge(df_vehicle_emergencies, on='Category', how='left')

In [9]:
# Preprocessing function
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [10]:
def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return " ".join(tokens)

In [11]:
df_combined['Processed_Query'] = df_combined['Query'].apply(preprocess_text)

In [12]:
df_combined.head()

,Category,Query,Response,Processed_Query
0,Flat Tire,What should I do if I get a flat tire?,Use your spare tire and jack to replace the fl...,get flat tire
1,Flat Tire,How do I change a flat tire?,Use your spare tire and jack to replace the fl...,change flat tire
2,Flat Tire,Can I drive with a flat tire?,Use your spare tire and jack to replace the fl...,drive flat tire
3,Flat Tire,What’s the quickest way to fix a flat tire?,Use your spare tire and jack to replace the fl...,quickest way fix flat tire
4,Flat Tire,Where can I get help for a flat tire?,Use your spare tire and jack to replace the fl...,get help flat tire


In [13]:
# Train-test split
X = df_combined['Processed_Query']
y = df_combined['Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Model pipeline
model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('rf', RandomForestClassifier())
])

In [15]:
# Train the model
model.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('rf', RandomForestClassifier())])

In [16]:
# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

                      precision    recall  f1-score   support

          Blown Fuse       1.00      1.00      1.00         4
       Brake Failure       1.00      1.00      1.00         4
     Car Won’t Start       0.50      1.00      0.67         1
        Dead Battery       0.00      0.00      0.00         1
  Engine Overheating       1.00      0.67      0.80         3
           Flat Tire       1.00      1.00      1.00         2
   Locked Out of Car       1.00      1.00      1.00         2
 Smoke from the Hood       0.67      1.00      0.80         2
Stuck in Mud or Snow       1.00      1.00      1.00         1

            accuracy                           0.90        20
           macro avg       0.80      0.85      0.81        20
        weighted avg       0.89      0.90      0.88        20



In [17]:
model.classes_

array(['Blown Fuse', 'Brake Failure', 'Car Won’t Start', 'Dead Battery',
       'Engine Overheating', 'Flat Tire', 'Fuel Leak',
       'Locked Out of Car', 'Smoke from the Hood', 'Stuck in Mud or Snow'],
      dtype=object)

In [18]:
def hybrid_chatbot(query, threshold=0.1):
    # Preprocess the query
    processed_query = preprocess_text(query)

    try:
        # Get prediction probabilities
        predicted_probs = model.predict_proba([processed_query])[0]

        # Get the max probability and corresponding category
        max_prob = np.max(predicted_probs)
        predicted_category = model.classes_[np.argmax(predicted_probs)]

        # Check against the threshold
        if max_prob < threshold:
            response = "I'm sorry, I couldn't understand your issue. Could you provide more details or rephrase?"
        else:
            # Retrieve response
            response = df_vehicle_emergencies.loc[
                df_vehicle_emergencies['Category'] == predicted_category,
                'Response'
            ].values[0]
    except:
        # Default response for any unexpected issues
        response = "I'm sorry, I couldn't understand your issue. Could you provide more details or rephrase?"

    return response

In [19]:
# Test the chatbot
user_query = "I think there is a leak"
response = hybrid_chatbot(user_query)
print("Chatbot Response:", response)

Chatbot Response: Turn off the engine immediately and move away from the vehicle to avoid any risk of fire.


In [20]:
import joblib

# Save the model to a file
joblib.dump(model, 'vehicle_assistance_model.joblib')

['vehicle_assistance_model.joblib']